In [11]:
# %%writefile app.py guarda todo el script del chatbot en un archivo app.py que luego puede ser ejecutado independientemente del notebook.
%%writefile app.py
import streamlit as st  # Crear interfaz web interactiva
import pandas as pd # Manejo de datasets
import numpy as np  # Operaciones numéricas
import torch # Procesamiento con redes neuronales
import nltk # Procesamiento de lenguaje natural
import pickle # Serializar y deserializar objetos (guardar/cargar modelo)
import os # Operaciones con sistema de archivos
from google.colab import drive # Montar y acceder a archivos en Google Drive
from transformers import AutoTokenizer, AutoModel  # Modelos de lenguaje BERT
from sklearn.feature_extraction.text import TfidfVectorizer #  Vectorización
from sklearn.metrics.pairwise import cosine_similarity # Calcular similitud semántica entre textos
from nltk.tokenize import word_tokenize # Dividir texto en tokens
from nltk.stem import SnowballStemmer # Reducir palabras a su raíz (stemming) en español
from nltk.corpus import stopwords # Obtener lista de palabras vacías (stopwords) en español
import re # Limpiar y manipular texto
from datetime import datetime # Manejar fechas y timestamps

# Descargar recursos NLTK
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('all')

class ChatbotAvanzado:
    def __init__(self, ruta_dataset, ruta_modelo=None):
        # Inicializar stemmer y stop_words siempre
        self.stemmer = SnowballStemmer('spanish')
        self.stop_words = set(stopwords.words('spanish'))

        # Carga o Creación de Modelo
        if ruta_modelo and self.cargar_modelo(ruta_modelo):
            st.success("Modelo cargado exitosamente.")
        else:
            st.info("Creando nuevo modelo...")
            self.df = pd.read_csv(ruta_dataset)

            # Preprocesar datos
            self.preprocesar_datos()

            # Vectorización (Prepara datos para cálculo de similitud semántica)
            self.vectorizador = TfidfVectorizer(
                stop_words=list(self.stop_words),
                max_features=5000
            )
            self.X = self.vectorizador.fit_transform(self.df['texto_limpio'])

        # Cargar modelo de lenguaje (siempre se carga porque no lo guardamos)
        self.tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
        self.model = AutoModel.from_pretrained("bert-base-multilingual-cased")

        # Contexto de conversación
        self.contexto = []

        # Nuevas variables para almacenar interacciones
        self.nuevas_interacciones = []
        self.umbral_actualizacion = 10  # Número de interacciones antes de actualizar

    def preprocesar_texto(self, texto):
        """Preprocesar texto"""
        texto = str(texto).lower()
        texto = re.sub(r'[^a-záéíóúñ\s]', '', texto)
        tokens = word_tokenize(texto)
        tokens = [self.stemmer.stem(word) for word in tokens if word not in self.stop_words]
        return ' '.join(tokens)

    def preprocesar_datos(self):
        """Preprocesar columnas del dataset"""
        self.df['texto_limpio'] = self.df['pregunta'].apply(self.preprocesar_texto)

    def obtener_embedding_bert(self, texto):
        """Obtener embedding con BERT"""
        inputs = self.tokenizer(texto, return_tensors="pt", padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = self.model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

    def buscar_respuesta_semantica(self, consulta):
      """Buscar respuesta usando similitud semántica"""
      consulta_limpia = self.preprocesar_texto(consulta)

      # Vectorización TF-IDF
      consulta_vectorizada = self.vectorizador.transform([consulta_limpia])
      similitudes_tfidf = cosine_similarity(consulta_vectorizada, self.X)[0]

      # Embedding BERT para similitud semántica
      consulta_embedding = self.obtener_embedding_bert(consulta)

      # Combinar métodos de similitud
      indices_top = np.argsort(similitudes_tfidf)[::-1][:5]

      mejores_respuestas = []
      for idx in indices_top:
          respuesta_candidata = self.df.iloc[idx]
          similitud_bert = cosine_similarity(
              [consulta_embedding],
              [self.obtener_embedding_bert(respuesta_candidata['pregunta'])]
          )[0][0]

          mejores_respuestas.append({
              'respuesta': respuesta_candidata['respuesta'],
              'similitud_tfidf': similitudes_tfidf[idx],
              'similitud_bert': similitud_bert
          })

      # Ordenar por una combinación de similitudes
      mejores_respuestas.sort(key=lambda x: (x['similitud_tfidf'] + x['similitud_bert']), reverse=True)

      umbral_confianza = 1.0  # Definir un umbral de confianza
      if mejores_respuestas and (mejores_respuestas[0]['similitud_tfidf'] + mejores_respuestas[0]['similitud_bert']) > umbral_confianza:
          mejor_respuesta = mejores_respuestas[0]['respuesta']
          mejor_similitud = mejores_respuestas[0]['similitud_tfidf'] + mejores_respuestas[0]['similitud_bert']
      else:
          mejor_respuesta = "Lo siento, no tengo una respuesta confiable para esa pregunta."
          mejor_similitud = mejores_respuestas[0]['similitud_tfidf'] + mejores_respuestas[0]['similitud_bert']

      return mejor_respuesta, mejor_similitud
    def manejar_contexto(self, consulta):
        """Manejar contexto de conversación"""
        self.contexto.append(consulta)
        if len(self.contexto) > 3:
            self.contexto.pop(0)

        return self.buscar_respuesta_semantica(consulta)

    def almacenar_interaccion(self, pregunta, respuesta, retroalimentacion):
        """Almacena una nueva interacción"""
        self.nuevas_interacciones.append({
            'pregunta': pregunta,
            'respuesta': respuesta,
            'retroalimentacion': retroalimentacion,
            'fecha': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        })

    def actualizar_modelo(self):
        """Actualiza el modelo con las nuevas interacciones"""
        if len(self.nuevas_interacciones) > 0:
            # Convertir interacciones a DataFrame
            nuevos_datos = pd.DataFrame(self.nuevas_interacciones)
            # Concatenar interacciones
            self.df = pd.concat([self.df, nuevos_datos[['pregunta', 'respuesta']]], ignore_index=True)
            # Reprocesar y actualizar vectorización
            self.preprocesar_datos()
            self.X = self.vectorizador.fit_transform(self.df['texto_limpio'])
            # Reiniciar interacciones
            self.nuevas_interacciones = []
            st.success("Modelo actualizado con nuevas interacciones.")

    def guardar_modelo(self, ruta_guardado):
        """Guarda el modelo y los datos procesados"""
        self.actualizar_modelo()  # Incluir las últimas interacciones
        datos_guardado = {
            'vectorizador': self.vectorizador,
            'X': self.X,
            'df': self.df,
            'nuevas_interacciones': self.nuevas_interacciones
        }
        with open(ruta_guardado, 'wb') as archivo:
            pickle.dump(datos_guardado, archivo)
        st.success(f"Modelo guardado en {ruta_guardado}")

    def cargar_modelo(self, ruta_carga):
        """Carga el modelo y los datos procesados"""
        if os.path.exists(ruta_carga):
            with open(ruta_carga, 'rb') as archivo:
                datos_cargados = pickle.load(archivo)
            self.vectorizador = datos_cargados['vectorizador']
            self.X = datos_cargados['X']
            self.df = datos_cargados['df']
            self.nuevas_interacciones = datos_cargados.get('nuevas_interacciones', [])
            return True
        return False

def main():
    st.title("🤖 Chatbot de Asistencia al Cliente - Gonzalo Cáceres")
    st.write("Bienvenido al asistente de soporte técnico. ¿En qué puedo ayudarte hoy?")

    # Rutas de archivos
    RUTA_DATASET = '/content/drive/My Drive/LLM/datos_chatbot_soporte_tecnico.csv'
    RUTA_MODELO = '/content/drive/My Drive/LLM/modelo_chatbot.pkl'

    # Inicializar chatbot
    if 'chatbot' not in st.session_state:
        st.session_state.chatbot = ChatbotAvanzado(RUTA_DATASET, RUTA_MODELO)

    # Inicializar historial de mensajes
    if 'messages' not in st.session_state:
        st.session_state.messages = []

    # Mostrar historial de mensajes
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    # Input del usuario
    if prompt := st.chat_input("Escribe tu pregunta aquí"):
        st.session_state.messages.append({"role": "user", "content": prompt})
        with st.chat_message("user"):
            st.markdown(prompt)

        respuesta, confianza = st.session_state.chatbot.manejar_contexto(prompt)
        st.session_state.messages.append({"role": "assistant", "content": respuesta})
        with st.chat_message("assistant"):
            st.markdown(respuesta)
            st.markdown(f"Confianza: {confianza:.2f}")

        # Retroalimentación del usuario
        retroalimentacion = st.slider("Califica la utilidad de la respuesta", 1, 5, 3)
        st.session_state.chatbot.almacenar_interaccion(prompt, respuesta, retroalimentacion)

        # Actualizar el modelo si se alcanza el umbral
        if len(st.session_state.chatbot.nuevas_interacciones) >= st.session_state.chatbot.umbral_actualizacion:
            st.session_state.chatbot.actualizar_modelo()
            st.session_state.chatbot.guardar_modelo(RUTA_MODELO)

if __name__ == "__main__":
    main()

Overwriting app.py


In [3]:
# Instalación de librerías
!pip install streamlit transformers torch scikit-learn pandas nltk

In [9]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!curl https://loca.lt/mytunnelpassword

34.80.175.23

In [12]:
!streamlit run app.py &>/content/logs.txt &
!npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹your url is: https://smart-lights-notice.loca.lt
^C
